In [ ]:
%matplotlib inline

In [3]:
from absa.config import DATA_PATHS
from absa.dataset import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import spacy
nlp = spacy.load('en')

In [4]:
train_ds_path = DATA_PATHS['asba.semeval16.raw.train']

df_train = load_dataset(train_ds_path)

df_train = pd.DataFrame({
    'text': df_train.groupby('id')['text'].first(),
    'categories': df_train.groupby('id')['category'].apply(list),
})

df_train.head()

,text,categories
id,,
1004293:0,Judging from previous posts this used to be a ...,[RESTAURANT#GENERAL]
1004293:1,"We, there were four of us, arrived at noon - t...",[SERVICE#GENERAL]
1004293:2,"They never brought us complimentary noodles, i...",[SERVICE#GENERAL]
1004293:3,The food was lousy - too sweet or too salty an...,"[FOOD#QUALITY, FOOD#STYLE_OPTIONS]"
1004293:4,"After all that, they complained to me about th...",[SERVICE#GENERAL]


In [5]:
lookup = {}
with open('../resources/vocabulary.txt', 'r', encoding='utf-8') as f:
    for idx, section in enumerate(f.read().split('\n\n')):
        for key in section.split('\n'):
            lookup[key.lower()] = idx
lookup

{'restaurant': 0,
 'bar': 0,
 'bill': 0,
 'booking': 0,
 'reservation': 0,
 'breakfast': 0,
 'chef': 0,
 'dinner': 0,
 'lunch': 0,
 'main course': 0,
 'menu': 1,
 'service charge': 0,
 'starter': 0,
 'tip': 0,
 'all day breakfast': 1,
 'apple crumble': 1,
 'apple pie': 2,
 'bacon and eggs': 1,
 'baked potato or jacket potato': 1,
 'beefburger or hamburger': 1,
 'boiled potatoes': 1,
 'boiled rice': 1,
 'bread and butter pudding': 2,
 'catch of the day': 1,
 'cauliflower cheese': 2,
 'cheese and biscuits': 1,
 'cheeseburger': 3,
 'cheesecake': 2,
 'chicken': 2,
 'chocolate cake': 2,
 'chocolate mousse': 1,
 'crème brûlée': 1,
 'curry': 2,
 'custard': 2,
 'danish pastry': 1,
 'dish of the day': 1,
 'egg and chips': 1,
 'fillet steak': 1,
 'fish and chips': 1,
 'fish pie': 2,
 'french fries or fries': 1,
 'fried rice': 1,
 'fruit salad': 2,
 'full english breakfast': 1,
 'green salad': 1,
 'ice cream': 2,
 'lasagne': 2,
 'lemon meringue pie': 2,
 'lemon tart': 1,
 'mashed potatoes': 1,
 '

In [6]:
def find_terms(sents):
    for sent in sents:
        term_keys = {}
        for key in lookup.keys():
            if key in sent.lower():
                if lookup[key] not in term_keys:
                    term_keys[lookup[key]] = 0
                term_keys[lookup[key]] += 1
        yield term_keys

list(find_terms(['Pizza was too hot!']))

[{2: 1, 4: 1}]

# Extract Features

In [7]:
def lemmatize(x):
    return ' '.join([token.lemma_ for token in nlp(x)])

In [8]:
pipeline = make_union(
    TfidfVectorizer(preprocessor=lemmatize, stop_words='english', ngram_range=(1, 3)),
    make_pipeline(
        FunctionTransformer(lambda x: list(find_terms(x))),
        DictVectorizer()
    ),
)

x_train = pipeline.fit_transform(df_train.text).toarray()

x_train.shape

C:\Users\prash\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['PRON', 'make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\prash\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


ValueError: could not convert string to float: 'Judging from previous posts this used to be a good place, but not any longer.'

# Prepare Labels

In [ ]:
mlb = MultiLabelBinarizer()

y_train = mlb.fit_transform(df_train.categories)

y_train.shape

# Cross Validatation

In [ ]:
scoring = ['precision_micro', 'recall_micro', 'f1_micro']

clf = OneVsRestClassifier(LogisticRegression())

scores = cross_validate(clf, x_train, y_train, scoring=scoring, cv=3)

print('Micro Precision Score\t', np.average(scores['test_precision_micro']))
print('Micro Recall Score\t', np.average(scores['test_recall_micro']))
print('Micro F1 Score\t\t', np.average(scores['test_f1_micro']))

# Train Model

In [ ]:
clf.fit(x_train, y_train)

# Evaluate Model

In [ ]:
test_ds_path = DATA_PATHS['asba.semeval16.raw.test.gold']

df_test = load_dataset(test_ds_path)

df_test = pd.DataFrame({
    'text': df_test.groupby('id')['text'].first(),
    'categories': df_test.groupby('id')['category'].apply(list),
})

df_test.head()

In [ ]:
# Testing
x_test = pipeline.transform(df_test.text).toarray()

y_pred = clf.predict(x_test)

y_pred.shape

In [ ]:
y_true = mlb.transform(df_test.categories)

y_true.shape

In [ ]:
('f1_score', f1_score(y_true, y_pred, average='micro'))